In [ ]:
!pip install datasets==2.14.6
!pip install transformers
!pip install evaluate
!pip install --no-cache-dir transformers sentencepiece
!pip install accelerate -U

In [ ]:
import torch
from datasets import Dataset, DatasetDict
from datasets import concatenate_datasets
from torch.utils.data import DataLoader
import os
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
from transformers import AutoTokenizer, AutoModelForMultipleChoice, get_scheduler, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import evaluate
import accelerate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model_name = "FacebookAI/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
train_data = np.load('/content/drive/MyDrive/data/SP-train.npy', allow_pickle=True)

In [ ]:
def convert_to_dataset_type(array):
    df = pd.DataFrame(array.tolist())
    col = ['id','distractor1','distractor2','distractor(unsure)']
    for c in col:
      df[c] = df[c].astype(str)
    df['label'] = df['label'].astype(int)
    data = Dataset.from_pandas(df, split = "train")
    return data

def preprocess_function(examples):
    first_sentences = [[context] * 4 for context in examples["question"]]
    first_sentences = sum(first_sentences, [])
    second_sentences = [item for item in examples["choice_list"]]
    second_sentences = sum(second_sentences, [])
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

In [ ]:
train_dataset = convert_to_dataset_type(train_data)
tokenized_train = train_dataset.map(preprocess_function, batched=True)
print(f"Training set size: {len(tokenized_train)}")

Map:   0%|          | 0/507 [00:00<?, ? examples/s]

Training set size: 507


In [ ]:
def split_dataset(dataset):
    train_dta, temp_dta = dataset.train_test_split(test_size = 0.2, shuffle=True)
    val_dta, test_dta = temp_dta.train_test_split(test_size = 0.5, shuffle=True)
    return DatasetDict({"train": train_dta,"test": test_dta,"valid": val_dta})

my_dataset = split_dataset(tokenized_train)
print(len(my_dataset['train']))
print(len(my_dataset['test']))
print(len(my_dataset['valid']))

In [ ]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
def get_final_dataset(dataset):
    tokenized_dataset = dataset.rename_column("label", "labels")
    tokenized_dataset = tokenized_dataset.remove_columns(['id', 'question', 'answer', 'distractor1', 'distractor2', 'distractor(unsure)', 'choice_list', 'choice_order'])
    tokenized_dataset.set_format("torch")
    return tokenized_dataset

tokenized_datasets = get_final_dataset(my_dataset)
os.environ["WANDB_DISABLED"] = "true"

model = AutoModelForMultipleChoice.from_pretrained(model_name, ignore_mismatched_sizes=True)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model.to(device)

Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


cuda


RobertaForMultipleChoice(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [ ]:
batch_size = 4
lr = 3e-5
num_epochs = 3

num_training_steps = (len(my_dataset["train"]) // batch_size) * num_epochs
batches_per_epoch = len(my_dataset["train"]) // batch_size

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
training_args = TrainingArguments(
    output_dir = "./output",
    evaluation_strategy="steps",
    eval_steps=20,
    logging_steps=20,
    logging_strategy="steps",
    learning_rate=lr,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    report_to=None,
    save_strategy="no"
    )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    optimizers=(optimizer, lr_scheduler),
    compute_metrics=compute_metrics
)

In [ ]:
print(f'Training {model_name}')
train_result = trainer.train()

Training FacebookAI/roberta-large


You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy
20,1.366700,1.364752,0.509804
40,1.074000,0.765653,0.666667
60,0.677100,1.178807,0.666667
80,0.901200,0.879480,0.627451
100,0.747400,0.653770,0.745098
120,0.359500,0.534894,0.823529
140,0.307300,0.499781,0.823529
160,0.351500,0.628856,0.803922
180,0.370400,0.492840,0.882353
200,0.316200,0.507408,0.901961


In [ ]:
metrics = train_result.metrics
trainer.save_metrics("train", metrics)
trainer.save_state()

In [ ]:
def evaluate_accuracy(dataset, pred_list=False):
    total_answers = 0
    correct_answers = 0
    predictions = []

    model.eval()

    for i in dataset:
        prompt = i['question'].strip()
        candidates = i['choice_list']
        true_label_original = i['label']
        candidate_1, candidate_2, candidate_3, candidate_4 = candidates[0].strip(), candidates[1].strip(), candidates[2].strip(), candidates[3].strip()

        inputs = tokenizer([[prompt, candidate_1], [prompt, candidate_2], [prompt, candidate_3], [prompt, candidate_4]],
                           return_tensors="pt", padding=True).to("cuda")

        labels = torch.tensor(true_label_original).unsqueeze(0).to("cuda")  # Batch size 1

        # Pass the input through the model to obtain predictions
        with torch.no_grad():
            outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)

        logits = outputs.logits
        predicted_class = logits.argmax().item()

        predictions.append(predicted_class)

        if predicted_class == true_label_original:
            correct_answers += 1
        total_answers += 1

    # Calculate accuracy
    accuracy = correct_answers / total_answers

    # Round accuracy to three decimal places
    rounded_accuracy = round(accuracy, 3)
    print("Accuracy is", rounded_accuracy)

    if pred_list:
        return rounded_accuracy, predictions
    return rounded_accuracy

In [ ]:
acc = evaluate_accuracy(my_dataset['test'])

Accuracy is 0.882
